# Task 1 — Data Pipeline Development

This notebook demonstrates an ETL pipeline using **Pandas** and **scikit-learn**. It loads the Iris dataset, introduces and imputes missing values, encodes a categorical column, scales numeric features, performs a train/test split, and saves the processed dataset and pipeline.

In [ ]:
# 1. Imports
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

print('Libraries imported')

In [ ]:
# 2. Load Iris dataset into a pandas DataFrame
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target, name='species')

# Create a human-readable species column to demonstrate categorical encoding
species_names = [iris.target_names[i] for i in y]
X['species_str'] = species_names

# Introduce some missing values for demonstration (set 5 random cells in numeric columns to NaN)
rng = np.random.default_rng(42)
rows = rng.choice(X.index, size=5, replace=False)
cols = rng.choice(X.columns[:-1], size=5, replace=True)  # numeric columns only
for r, c in zip(rows, cols):
    X.loc[r, c] = np.nan

print('Loaded DataFrame with shape:', X.shape)
X.head()

In [ ]:
# Display a small preview for the user (this uses the notebook display helper provided by the environment)
import caas_jupyter_tools as cjt
cjt.display_dataframe_to_user('ETL Sample - raw', X.head(10))

# Also show the target
print('\nTarget distribution:\n', y.value_counts())

In [ ]:
# 3. ETL steps: Impute, encode categorical, scale numeric
numeric_features = iris.feature_names
categorical_features = ['species_str']

# Imputer for numeric
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Encoder for categorical
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Full pipeline object (useful to save and reuse)
etl_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Fit-transform the data
X_processed = etl_pipeline.fit_transform(X)
print('Processed array shape:', X_processed.shape)

In [ ]:
# 4. Decode feature names after ColumnTransformer for a nicer DataFrame
# Get numeric column names (they stay the same)
num_cols = numeric_features
# Get categorical one-hot names
cat_cols = list(etl_pipeline.named_steps['preprocessor']
                .named_transformers_['cat']
                .named_steps['onehot']
                .get_feature_names_out(categorical_features))

processed_columns = list(num_cols) + cat_cols

# Create DataFrame
X_processed_df = pd.DataFrame(X_processed, columns=processed_columns)

# Attach the numeric target for convenience
X_processed_df['species'] = y.values

print('Processed DataFrame preview:')
X_processed_df.head()

In [ ]:
# Show processed data to user
cjt.display_dataframe_to_user('ETL Sample - processed', X_processed_df.head(10))

# 5. Train/test split and save processed CSV and pipeline
X_train, X_test, y_train, y_test = train_test_split(X_processed_df.drop(columns=['species']), X_processed_df['species'], test_size=0.2, random_state=42)

print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)

# Save processed CSVs
train_path = '/mnt/data/processed_train.csv'
test_path = '/mnt/data/processed_test.csv'
X_train.join(y_train).to_csv(train_path, index=False)
X_test.join(y_test).to_csv(test_path, index=False)

# Save the fitted ETL pipeline
pipeline_path = '/mnt/data/etl_pipeline.joblib'
joblib.dump(etl_pipeline, pipeline_path)

print('Saved processed_train.csv, processed_test.csv, and etl_pipeline.joblib to /mnt/data')

In [ ]:
# 6. Quick checks: load back pipeline and transform a new sample
loaded_pipeline = joblib.load('/mnt/data/etl_pipeline.joblib')
# take the first raw sample and show transformation
raw_sample = X.iloc[[0]]
print('Raw sample:\n', raw_sample)
transformed_sample = loaded_pipeline.transform(raw_sample)
print('\nTransformed sample shape:', transformed_sample.shape)

# Convert to DataFrame for readability
transformed_df = pd.DataFrame(transformed_sample, columns=processed_columns)
transformed_df

## How to run this notebook

1. Open this notebook in Jupyter or JupyterLab.
2. Run cells sequentially. The notebook uses only built-in sklearn datasets so no external downloads are required.
3. Output files are saved to `/mnt/data/`:
- `processed_train.csv`
- `processed_test.csv`
- `etl_pipeline.joblib`

You can replace the Iris dataset with any CSV by loading it with `pd.read_csv()` and adapting the `numeric_features` and `categorical_features` lists accordingly.

If you'd like, I can also create a short README and a Python script version of this pipeline.